In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from shapely.geometry import Point
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/Dane z Kaggla/GDAL/SHP/Wind_Turbine")
!ls

distances_above_700m.csv  Wind_Turbine_cleaned.prj  Wind_Turbine.cpg  Wind_Turbine.shp
Wind_Turbine_cleaned.cpg  Wind_Turbine_cleaned.shp  Wind_Turbine.dbf  Wind_Turbine.shx
Wind_Turbine_cleaned.dbf  Wind_Turbine_cleaned.shx  Wind_Turbine.prj


In [ ]:
src_Path = '/content/drive/MyDrive/Dane z Kaggla/GDAL/SHP/Wind_Turbine'

In [ ]:
input_shp_files = [r for r in os.listdir(src_Path) if r.endswith('_cleaned.shp')]
input_shp_files

['Wind_Turbine_cleaned.shp']

In [ ]:
output_shp_file = []
for srcRst in input_shp_files:
  dstRst = 'distance_' + srcRst[:-4] +srcRst[-4:]
  output_shp_file.append(dstRst)
output_shp_file

['distance_Wind_Turbine_cleaned.shp']

In [ ]:
import os
import numpy as np
import geopandas as gpd
from scipy.spatial import distance_matrix

def filter_and_check_shapefiles(shapefile_names, distance_threshold=700):
    """
    Load multiple shapefiles, calculate distances between points, add a distance column,
    filter points by distance, and save the filtered shapefiles. Additionally, save points
    that have any other point within a distance shorter than the threshold to a separate shapefile.

    Parameters:
    shapefile_names (list of str): List of shapefile names.
    distance_threshold (float): Distance threshold in meters.
    """
    for shapefile_name in shapefile_names:
        # Load the shapefile
        gdf = gpd.read_file(shapefile_name)

        # Calculate the distance matrix
        coords = np.array(list(gdf.geometry.apply(lambda geom: (geom.x, geom.y))))
        dist_matrix = distance_matrix(coords, coords)

        # Add a column with the minimum distance to the closest point
        np.fill_diagonal(dist_matrix, np.inf)  # Ignore self-distance by setting diagonal to infinity
        min_distances = dist_matrix.min(axis=1)
        gdf['min_distance'] = min_distances

        # Identify points with any other point within the distance threshold
        close_points_indices = np.any(dist_matrix < distance_threshold, axis=1)

        # Create a GeoDataFrame for points to check
        points_to_check = gdf[close_points_indices]

        # Remove points that are closer than the threshold, keeping the first occurrence
        gdf_filtered = gdf[~close_points_indices]

        # Construct output file names
        base_name = os.path.splitext(shapefile_name)[0]
        output_filtered_shapefile = f"{base_name}_filtered.shp"
        output_points_to_check_shapefile = f"{base_name}_points_to_check.shp"

        # Save the filtered shapefile
        gdf_filtered.to_file(output_filtered_shapefile, driver='ESRI Shapefile')
        print(f"Filtered shapefile saved to {output_filtered_shapefile}")

        # Save the points to check shapefile
        points_to_check.to_file(output_points_to_check_shapefile, driver='ESRI Shapefile')
        print(f"Points to check shapefile saved to {output_points_to_check_shapefile}")


filter_and_check_shapefiles(input_shp_files)
